In [1]:
from astropy.table import Table
import numpy as np

for quasar data, used query:
```sql
SELECT t.source_id, t.ra, t.dec
FROM gaiadr3.gaia_source AS t, gaiadr3.qso_candidates AS i
WHERE t.source_id = i.source_id AND t.dec < 6.0 AND t.dec > -3.0 AND t.ra > 125 AND t.ra < 148 AND i.classlabel_dsc = 'quasar'
```

In [2]:
quasar_data = Table.read('Quasar_data.csv')
cluster_data = Table.read('cluster_data.csv')

In [3]:
cluster_data[1]

ID,ID_SRC,RA,DEC,EXT_LIKE,DET_LIKE,z,z_type,T_300kpc,T_300kpc_L,T_300kpc_U,L_300kpc,L_300kpc_L,L_300kpc_U,Lbol_300kpc,Lbol_300kpc_L,Lbol_300kpc_U,F_300kpc,F_300kpc_L,F_300kpc_U,T_500kpc,T_500kpc_L,T_500kpc_U,L_500kpc,L_500kpc_L,L_500kpc_U,Lbol_500kpc,Lbol_500kpc_L,Lbol_500kpc_U,F_500kpc,F_500kpc_L,F_500kpc_U,SNR_MAX,R_SNR_MAX
str22,int32,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
eFEDS J082751.7-002853,11248,126.96547052351899,-0.4816375760468516,12.791595,27.86591,0.25716,0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0191e+43,-1.0,-1.0,1.7219e+43,-1.0,-1.0,4.7107e-14,-1.0,-1.0,-1.0,-1.0,-1.0,1.0611e+43,-1.0,-1.0,1.7963e+43,-1.0,-1.0,4.9033e-14,3.07,0.7393


In [4]:
quasar_data[1]

source_id,ra,dec
str19,float64,float64
3090033260698985472,125.20060237716548,1.27928355298535


In [5]:
quasar_data['ra_rad'] = np.radians(quasar_data['ra'])
quasar_data['dec_rad'] = np.radians(quasar_data['dec'])

In [6]:

matches_found = 0
five_arc_minutes = 0.08333333 #degrees
five_arc_seconds = 0.00138889 #degrees
rejection_list= [1644,3334,8602,5909,8922,9463,13484,13299,11754,5702,6840]
rejected = 0
for cluster in cluster_data:
    a_b = np.radians(cluster['RA'])
    d_b = np.radians(cluster['DEC'])

    angles = np.arccos(np.sin(quasar_data['dec_rad'])*np.sin(d_b) + np.cos(quasar_data['dec_rad'])*np.cos(d_b)*np.cos(quasar_data['ra_rad'] - a_b))
    matches = angles < np.radians(five_arc_seconds)
    count = np.count_nonzero(matches)
    if count > 0 :
        matched_quasars = quasar_data[matches]
        for quasar in matched_quasars:
            if not cluster['ID_SRC'] in rejection_list:
                print('eFEDS ID : ',cluster['ID_SRC'], ' Matched to quasar ID: ', quasar['source_id'])
                matches_found += 1
            else:
                print('rejected : ','eFEDS ID : ', cluster['ID_SRC'], ' Matched to quasar ID: ', quasar['source_id'])
                rejected += 1

    # for quasar in quasar_data:
    #     a_a = np.radians(quasar['ra'])        
    #     d_a = np.radians(quasar['dec'])        
    #     angle = np.arccos(np.sin(d_a)*np.sin(d_b) + np.cos(d_a)*np.cos(d_b)*np.cos(a_a - a_b))
    #     if angle < np.radians(five_arc_minutes):
    #         if not cluster['ID_SRC'] in rejection_list:
    #             print('eFEDS ID : ',cluster['ID_SRC'], ' Matched to quasar ID: ', quasar['source_id'])
    #             matches_found += 1
    #         else:
    #             print('rejected : ','eFEDS ID : ', cluster['ID_SRC'], ' Matched to quasar ID: ', quasar['source_id'])
    #             rejected += 1
            

print(matches_found)
print(rejected)

eFEDS ID :  3198  Matched to quasar ID:  3080001930786796544
eFEDS ID :  7902  Matched to quasar ID:  3079964409952469120
eFEDS ID :  4348  Matched to quasar ID:  580153157163265152
eFEDS ID :  1527  Matched to quasar ID:  3841007360398241536
eFEDS ID :  3773  Matched to quasar ID:  3850846340196281344
5
0


In [7]:
# fake_ra = np.random.uniform(np.min(quasar_data['ra_rad']), np.max(quasar_data['ra_rad']), len(quasar_data))
# fake_dec = np.random.uniform(np.min(quasar_data['dec_rad']), np.max(quasar_data['dec_rad']), len(quasar_data))
count = 0
for i in range(100):
    fake_ra = np.radians(np.random.uniform(125,148,len(quasar_data)))
    fake_dec = np.radians(np.random.uniform(-3,6,len(quasar_data)))
    for cluster in cluster_data:
        a_b = np.radians(cluster['RA'])
        d_b = np.radians(cluster['DEC'])
        angles = np.arccos(np.sin(fake_dec)*np.sin(d_b) + np.cos(fake_dec)*np.cos(d_b)*np.cos(fake_ra - a_b))
        count += np.count_nonzero(angles < np.radians(five_arc_seconds))

print(count/100)

0.24
